# Геолокация сообщений

#### Импорт библиотек

In [1]:
import warnings

warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sloyka.src.geocoder.geocoder import Geocoder

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to /Users/test/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/test/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2024-06-27 15:34:03,808 SequenceTagger predicts: Dictionary with 7 tags: O, S-Service, B-Service, E-Service, I-Service, <START>, <STOP>


#### Загрузка данных

In [2]:
import pandas as pd

s_data = {
        "text": [
            "На биржевой 15 снова шумят!!"
        ]
    }
df = pd.DataFrame(s_data)

In [3]:
# df = pd.read_csv('/Users/test/Documents/code/sloyka/sloyka/sample_data/vnukovomos.csv', sep=';')

In [4]:
df

,text
0,У Рубинштейна 25 дом дворовую территорию убери...
1,На биржевой 15 снова шумят!!


In [5]:
osm_id = 337422
geocoder = Geocoder(osm_id=osm_id, city_tags={'place':['state']})


2024-06-27 15:34:04,935 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS


Processing category place:   0%|          | 0/1 [00:00<?, ?it/s]/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pyproj/transformer.py:820: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return self._transformer._transform_point(
/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pyproj/transformer.py:820: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return self._transformer._transform_point(
Processing category place: 100%|██████████| 1/1 [00:00<00:00,  3.65it/s]
/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/py

In [6]:
result = geocoder.run(df, search_for_objects=True)
display(result)

100%|██████████| 2/2 [00:00<00:00, 162.18it/s]

06-27 15:34 | WARNING  | Error in extract_toponym with text 'На биржевой 15 снова шумят!!' and street_name 'биржевой': list index out of range
06-27 15:34 | INFO     | get_stem started


06-27 15:34 | INFO     | create_gdf started


100%|██████████| 4/4 [00:02<00:00,  1.98it/s]
/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/Users/test/Library/Caches/pypoetry/virtualenvs/sloyka-hdx_c6ud-py3.11/lib/python3.11/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_com

,text,Street,Numbers,Toponyms,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,other_geo_obj,geo_obj_tag
0,У Рубинштейна 25 дом дворовую территорию убери...,рубинштейна,,None,улица Рубинштейна Санкт-Петербург Россия,[улица Рубинштейна Санкт-Петербург Россия],улица Рубинштейна Санкт-Петербург Россия,улица Рубинштейна,"улица Рубинштейна, Владимирский округ, Санкт-П...",POINT (30.34356 59.92829),NaN,street
1,На биржевой 15 снова шумят!!,биржевой,15,None,"Биржевой переулок 15 Санкт-Петербург Россия,Би...","[Биржевой переулок 15 Санкт-Петербург Россия, ...",Биржевой переулок 15 Санкт-Петербург Россия,Биржевой переулок,"Биржевой переулок, округ № 7, Санкт-Петербург,...",POINT (30.29380 59.94515),NaN,street
2,На биржевой 15 снова шумят!!,биржевой,15,None,"Биржевой переулок 15 Санкт-Петербург Россия,Би...","[Биржевой переулок 15 Санкт-Петербург Россия, ...",Биржевой мост 15 Санкт-Петербург Россия,Биржевой мост,"Биржевой мост, Введенский округ, Санкт-Петербу...",POINT (30.30346 59.94736),NaN,street
3,На биржевой 15 снова шумят!!,биржевой,15,None,"Биржевой переулок 15 Санкт-Петербург Россия,Би...","[Биржевой переулок 15 Санкт-Петербург Россия, ...",Биржевой проезд 15 Санкт-Петербург Россия,Биржевой проезд,"Биржевой проезд, округ № 7, Санкт-Петербург, С...",POINT (30.30556 59.94319),NaN,street


In [7]:
result

,text,Street,Numbers,Toponyms,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,other_geo_obj,geo_obj_tag
0,У Рубинштейна 25 дом дворовую территорию убери...,рубинштейна,,None,улица Рубинштейна Санкт-Петербург Россия,[улица Рубинштейна Санкт-Петербург Россия],улица Рубинштейна Санкт-Петербург Россия,улица Рубинштейна,"улица Рубинштейна, Владимирский округ, Санкт-П...",POINT (30.34356 59.92829),NaN,street
1,На биржевой 15 снова шумят!!,биржевой,15,None,"Биржевой переулок 15 Санкт-Петербург Россия,Би...","[Биржевой переулок 15 Санкт-Петербург Россия, ...",Биржевой переулок 15 Санкт-Петербург Россия,Биржевой переулок,"Биржевой переулок, округ № 7, Санкт-Петербург,...",POINT (30.29380 59.94515),NaN,street
2,На биржевой 15 снова шумят!!,биржевой,15,None,"Биржевой переулок 15 Санкт-Петербург Россия,Би...","[Биржевой переулок 15 Санкт-Петербург Россия, ...",Биржевой мост 15 Санкт-Петербург Россия,Биржевой мост,"Биржевой мост, Введенский округ, Санкт-Петербу...",POINT (30.30346 59.94736),NaN,street
3,На биржевой 15 снова шумят!!,биржевой,15,None,"Биржевой переулок 15 Санкт-Петербург Россия,Би...","[Биржевой переулок 15 Санкт-Петербург Россия, ...",Биржевой проезд 15 Санкт-Петербург Россия,Биржевой проезд,"Биржевой проезд, округ № 7, Санкт-Петербург, С...",POINT (30.30556 59.94319),NaN,street


In [8]:
result.drop(columns=['Location']).iloc[:,:].explore()